In [8]:
import os
import requests
import pandas as pd
from tqdm import tqdm
import concurrent.futures



In [9]:
videos_url_csv_path = '/home/bfrisque/code/benoitfrisque/signlens/processed_data/videos_learning_platform.csv'

df = pd.read_csv(videos_url_csv_path)
df

,sign,url
0,book,https://media.asldeafined.com/vocabulary/14666...
1,book,https://www.signingsavvy.com/signs/mp4/14/1432...
2,book,https://s3-us-west-1.amazonaws.com/files.start...
3,book,http://aslbricks.org/New/ASL-Videos/book.mp4
4,book,http://www.aslsearch.com/signs/videos/book.mp4
...,...,...
1471,penny,http://www.aslsearch.com/signs/videos/one-cent...
1472,stuck,http://www.aslsearch.com/signs/videos/stuck.mp4
1473,stuck,https://media.asldeafined.com/vocabulary/14688...
1474,stuck,https://www.signingsavvy.com/signs/mp4/8/8099.mp4


In [10]:
def check_url(url):
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200, url
    except requests.RequestException:
        return False, url

In [11]:
# Check valid url  
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    future_to_url = {executor.submit(check_url, url): url for url in df['url']}
    for future in tqdm(concurrent.futures.as_completed(future_to_url), total=len(future_to_url), desc="Checking URLs"):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            if data[0]:
                df.loc[df['url'] == data[1], 'valid'] = True


Checking URLs:  99%|██████████████████████████████████████████████████████████████████████████▍| 1466/1476 [01:46<00:00, 13.79it/s]

KeyboardInterrupt



In [14]:
df.valid.value_counts() / len(df)

valid
True    0.821816
Name: count, dtype: float64

In [15]:
df = df[df['valid'] == True].reset_index(drop=True)

In [16]:
videos_url_csv_path_new = '/home/bfrisque/code/benoitfrisque/signlens/processed_data/videos_learning_platform_new.csv'

df.to_csv(videos_url_csv_path_new)